In [158]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import keras_tuner as kt
import cv2
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
from imblearn.combine import SMOTEENN



In [135]:
!pip install -q -U keras-tuner

In [144]:
# Initialize MediaPipe face mesh
mp_face_mesh = mp.solutions.face_mesh

class FER2013Dataset():
    def __init__(self, root_dir, transform = None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.label_map = {
            "angry": 0,
            "disgust": 1,
            "fear": 2,
            "happy": 3,
            "neutral": 4,
            "sad": 5,
            "surprise": 6
        }  # Mapping emotions to numerical labels  
   
   
   # Function to load and preprocess images (convert to grayscale, resize, normalize)
    def load_images_and_labels(self, directory, image_size=(48, 48)):
        images = []
        labels = []
    
        for emotion in os.listdir(directory):
            emotion_folder = os.path.join(directory, emotion)
            print(f"Emotion Folder: {emotion_folder}")

            if os.path.isdir(emotion_folder):
                # Get the label for the emotion based on the folder name
                label = self.label_map.get(emotion, None)


                # Ensure we have a valid label before proceeding
                if label is not None:
                    for filename in os.listdir(emotion_folder):
                        print(f"Processing file: {filename}")
                        image_path = os.path.join(emotion_folder, filename)

                        if filename.endswith(('.jpg')):  # Ensure it's an image file
                            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                            if image is None:
                                print(f"Could not load image: {image_path}")
                                continue  # Skip invalid images

            # Read and preprocess the image
            #image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
            image_resized = cv2.resize(image, image_size)  # Resize to 48x48
            image_normalized = image_resized / 255.0  # Normalize to range [0, 1]
            #image_normalized = np.expand_dims(image_normalized, axis=-1)  # Add grayscale channel
            
            images.append(image_normalized)
            labels.append(label)
    
        return np.array(images), np.array(labels)

# Load training and test data
train_dir = r'C:\Users\USER\Desktop\Final_Projects\.venv\Emotion_Detection\FER2013FolderDataset\train'
test_dir = r'C:\Users\USER\Desktop\Final_Projects\.venv\Emotion_Detection\FER2013FolderDataset\test'  # Path to your test folder


# Create an instance of the dataset class
dataset = FER2013Dataset(train_dir)


# Load the images and labels
X_train_images, y_train_labels = dataset.load_images_and_labels(train_dir)
X_test_images, y_test_labels = dataset.load_images_and_labels(test_dir)

# After loading the dataset, print out the unique labels to verify the data
X_train_images, y_train_labels = dataset.load_images_and_labels(train_dir)

print("Unique labels in y_train_labels:", np.unique(y_train_labels))


# Normalize the pixel values
X_train_images = X_train_images / 255.0
X_test_images = X_test_images / 255.0

X_train_images = X_train_images.reshape(-1, 48, 48, 1)
X_test_images = X_test_images.reshape(-1, 48, 48, 1)

# Assuming X_train_images is (num_samples, height, width, channels)
# Step 1: Flatten the image data
X_train_flattened = X_train_images.reshape(X_train_images.shape[0], -1)


# Check the distribution of classes
unique_classes, class_counts = np.unique(y_train_labels, return_counts=True)
print(f"Classes: {unique_classes}")
print(f"Counts: {class_counts}")

# Ensure you have more than one class
if len(unique_classes) <= 1:
    raise ValueError(f"Cannot apply SMOTEENN because there is only one class in the target labels: {unique_classes}")
  
# Step 1: Flatten the image data to 2D (samples, height * width * channels)
X_train_flattened = X_train_images.reshape(X_train_images.shape[0], -1)

# Step 2: Apply SMOTEENN (now the data is 2D, which SMOTEENN expects)
smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_train_flattened, y_train_labels)

# Step 3: Reshape the data back to 4D after resampling
X_resampled_reshaped = X_resampled.reshape(X_resampled.shape[0], 48, 48, 1)
y_resampled_reshaped = y_resampled.reshape(y_resampled.shape[0],48,48,1)


print("Resampling complete. New dataset shape:", X_resampled_reshaped.shape)

#unique_classes, class_counts = np.unique(y_train_labels, return_counts=True)
#print(f"Classes: {unique_classes}")
#print(f"Counts: {class_counts}")

# Data Augumentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')



# Assuming there are 7 classes in total
#y_test_labels = to_categorical(y_test_labels, num_classes=7)


# Create a generator for the training set
train_generator = datagen.flow(X_resampled_reshaped, y_resampled_reshaped, batch_size=32)
#test_generator = (X_test_images, y_test_labels)  # No augmentation for test data

# Create a generator for the test set without augmentation
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow(X_test_images, y_test_labels, batch_size=32)

# Ensure y_test_labels is in the correct format
if len(y_test_labels.shape) == 1:
    y_test_labels = to_categorical(y_test_labels, num_classes=7)

# Fit the model using the generator
#history = model.fit(train_generator, epochs=5, validation_data=test_generator, callbacks=[stop_early])



Emotion Folder: C:\Users\USER\Desktop\Final_Projects\.venv\Emotion_Detection\FER2013FolderDataset\train\angry
Processing file: Training_10118481.jpg
Processing file: Training_10120469.jpg
Processing file: Training_10131352.jpg
Processing file: Training_10161559.jpg
Processing file: Training_1021836.jpg
Processing file: Training_10269675.jpg
Processing file: Training_10278738.jpg
Processing file: Training_10290703.jpg
Processing file: Training_10295477.jpg
Processing file: Training_10315441.jpg
Processing file: Training_10316849.jpg
Processing file: Training_10333072.jpg
Processing file: Training_10334355.jpg
Processing file: Training_10345473.jpg
Processing file: Training_10422050.jpg
Processing file: Training_10485618.jpg
Processing file: Training_10503476.jpg
Processing file: Training_10524198.jpg
Processing file: Training_10539399.jpg
Processing file: Training_10592361.jpg
Processing file: Training_10595751.jpg
Processing file: Training_10608067.jpg
Processing file: Training_1074035

In [145]:
def model_builder(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape = (48, 48, 1)))

    hp_activation = hp.Choice('activation', values = ['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value = 16, max_value = 1000, step = 100)
    hp_layer_2 = hp.Int('layer_2', min_value = 16, max_value = 1000, step = 100)
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

    model.add(tf.keras.layers.Dense(units = hp_layer_1, activation = hp_activation))
    model.add(tf.keras.layers.BatchNormalization())  # Batch Normalization
    model.add(tf.keras.layers.Dense(units = hp_layer_2, activation = hp_activation))
    model.add(tf.keras.layers.BatchNormalization())  # Batch Normalization
    model.add(tf.keras.layers.Dropout(0.5))  # Drops 50% of neurons
    model.add(tf.keras.layers.Dense(64, kernel_regularizer=l2(0.001), activation='relu'))
    model.add(tf.keras.layers.Dense(7, activation = 'softmax'))
    

    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = hp_learning_rate),
                  loss = tf.keras.losses.sparse_categorical_crossentropy,
                  metrics = ['accuracy'])
    return model

In [146]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'fer2013')

Reloading Tuner from my_dir\fer2013\tuner0.json


In [147]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3, restore_best_weights=True)


In [148]:


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)


In [149]:
tuner.search(train_generator, epochs = 50, validation_split = 0.2, callbacks = [stop_early])

In [150]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

In [151]:
model = tuner.hypermodel.build(best_hps)
#history = model.fit(X_train_images, y_train_labels, epochs = 50, validation_split = 0.2, callbacks = [stop_early, reduce lr])


# Fit the model using the generator
history = model.fit(train_generator, epochs=5, validation_data=test_generator,callbacks = [stop_early])

c:\Users\USER\Desktop\Final_Projects\.venv\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Must provide at least one structure

In [152]:
import pandas as pd

pd.DataFrame(history.history)

,accuracy,loss,val_accuracy,val_loss
0,0.965626,0.211516,1.0,0.247691
1,1.000000,0.080093,1.0,0.103694
2,1.000000,0.067955,1.0,0.071473
3,1.000000,0.058316,1.0,0.057616
4,1.000000,0.049513,1.0,0.046751


In [153]:
model

<Sequential name=sequential_11, built=True>

In [155]:
model.evaluate(test_generator)

c:\Users\USER\Desktop\Final_Projects\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [156]:
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_11 (Flatten)            │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 401)            │       924,305 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 401)            │         1,604 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 601)            │       241,602 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 601)            │         2,404 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 601)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 64)             │        38,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,208,898 (4.61 MB)

 Trainable params: 1,206,894 (4.60 MB)

 Non-trainable params: 2,004 (7.83 KB)

In [159]:
model.save('MY_CNN_Model.h5')

In [54]:
model.save('my_model.keras')